![Logo BV IBMEC](https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/logo-bv-ibmec-notebooks.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ian-iania/IBMEC-BV-Modelos-Preditivos/blob/main/notebooks/NB02A_Markov_Buckets.ipynb)

# NB02A — Markov Buckets: Migração e PDD Projetada (6 meses)

- construir matriz de transição P a partir do histórico de buckets
- projetar distribuição por bucket para 6 meses
- converter em PDD em R$ e rodar stress test

## 1) Contexto FP&A

### O que vamos fazer
Ler buckets de atraso como estados e estimar uma matriz de migração mensal.

### Por que isso importa em FP&A
Com essa matriz, projetamos deterioração da carteira e impacto em provisão (PDD) no curto prazo.

### O que observar no output
- buckets são estados (`OK`, `Aten`, `Mau`, `WriteOff`)
- Markov aqui significa aplicar a matriz **mês a mês**
- o foco final é PDD projetada em R$

### 1.1) Imports da aula

### O que vamos fazer
Carregar bibliotecas para dados e visualização.

### Por que isso importa em FP&A
São os blocos mínimos para gerar matriz de migração, projeção e gráficos executivos.

### O que observar no output
A célula só prepara o ambiente.

In [ ]:
import pandas as pd  # manipulação de tabelas e datas
import numpy as np  # operações numéricas e projeções
import plotly.express as px  # gráficos interativos rápidos
import plotly.graph_objects as go  # gráficos customizados para comparação

### 1.2) Configuração do notebook

### O que vamos fazer
Definir seed e opções de visualização do pandas.

### Por que isso importa em FP&A
Garante reprodutibilidade e leitura limpa em sala.

### O que observar no output
Sem saída relevante, apenas configuração.

In [ ]:
np.random.seed(42)  # fixa seed para reprodutibilidade
pd.set_option("display.max_columns", 50)  # amplia número de colunas exibidas
pd.set_option("display.width", 140)  # melhora largura da tabela no notebook

## 2) Carregar dataset e sanity check

### O que vamos fazer
Ler o dataset de contratos mensais e ordenar por contrato/mês.

### Por que isso importa em FP&A
A construção de transições exige ordem temporal correta por contrato.

### O que observar no output
Base carregada com `month` em datetime e ordem consistente.

In [ ]:
url = "https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/data/bv_markov_buckets_contratos_mensal.csv"  # URL oficial do dataset

df = pd.read_csv(url, parse_dates=["month"])  # lê CSV e converte mês para data
df = df.sort_values(["contract_id", "month"]).reset_index(drop=True)  # ordena por contrato e tempo

### 2.1) Sanidade mínima da base

### O que vamos fazer
Ver tamanho, quantidade de contratos e quantidade de meses.

### Por que isso importa em FP&A
Garante que a base cobre histórico suficiente para montar migrações.

### O que observar no output
Números curtos e legíveis para validar volume e periodicidade.

In [ ]:
print(f"Shape: {df.shape[0]} linhas x {df.shape[1]} colunas")  # imprime tamanho da base
print(f"Contratos únicos: {df['contract_id'].nunique():,}")  # conta contratos distintos
print(f"Meses únicos: {df['month'].nunique()}")  # conta meses disponíveis

months_sorted = sorted(df["month"].unique())  # ordena meses para referência
print(f"Período: {months_sorted[0].date()} até {months_sorted[-1].date()}")  # imprime janela temporal

### 2.2) Distribuição de buckets no último mês

### O que vamos fazer
Pegar o último mês observado e mostrar participação por bucket.

### Por que isso importa em FP&A
Esse retrato é nosso ponto de partida para projeção (`π0`).

### O que observar no output
Participação relativa dos buckets no fim do histórico.

In [ ]:
states = ["OK", "Aten", "Mau", "WriteOff"]  # define ordem fixa dos estados
last_month = df["month"].max()  # identifica último mês disponível

df_last = df[df["month"] == last_month].copy()  # filtra apenas último mês
bucket_last = df_last["bucket"].value_counts().reindex(states, fill_value=0).reset_index()  # conta buckets em ordem fixa
bucket_last.columns = ["bucket", "qtd"]  # renomeia colunas da tabela
bucket_last["pct"] = bucket_last["qtd"] / bucket_last["qtd"].sum()  # calcula percentual por bucket

bucket_last  # exibe tabela base do último mês

### 2.3) Gráfico da distribuição no último mês (Plotly)

### O que vamos fazer
Visualizar em barras a participação de cada bucket no mês final.

### Por que isso importa em FP&A
Facilita discutir ponto de partida da carteira antes de projetar.

### O que observar no output
Peso de `OK` versus buckets críticos (`Mau` e `WriteOff`).

In [ ]:
fig_last = px.bar(  # cria gráfico de barras dos buckets no último mês
    bucket_last,
    x="bucket",
    y="pct",
    text=bucket_last["pct"].map(lambda x: f"{x:.1%}"),
    title=f"PARTE A — Distribuição de buckets no último mês ({last_month.date()})"
)
fig_last.update_layout(xaxis_title="Bucket", yaxis_title="Participação")  # ajusta títulos dos eixos
fig_last.show()  # renderiza gráfico interativo

## 3) Construir transições (bucket_t → bucket_t+1)

### O que vamos fazer
Criar `bucket_next` com `shift(-1)` dentro de cada contrato.

### Por que isso importa em FP&A
Sem esse passo não existe matriz de migração mensal.

### O que observar no output
Cada linha mostra estado atual e próximo estado do mesmo contrato.

In [ ]:
df_trans = df[["contract_id", "month", "bucket"]].copy()  # seleciona colunas necessárias para transição

df_trans["bucket_next"] = df_trans.groupby("contract_id")["bucket"].shift(-1)  # cria bucket do mês seguinte por contrato
df_trans = df_trans.dropna(subset=["bucket_next"]).copy()  # remove última observação sem próximo estado

### 3.1) Exemplo de transições

### O que vamos fazer
Exibir 5 linhas para fixar a leitura de `bucket` e `bucket_next`.

### Por que isso importa em FP&A
Ajuda a turma a enxergar como nasce a matriz P.

### O que observar no output
Movimentos como `OK→OK`, `OK→Aten`, `Mau→WriteOff`.

In [ ]:
df_trans[["contract_id", "month", "bucket", "bucket_next"]].head(5)  # mostra cinco transições reais da base

## 4) Estimar matriz de transição P

### O que vamos fazer
Contar transições e transformar em probabilidades por linha.

### Por que isso importa em FP&A
A matriz P resume dinâmica mensal dos buckets.

### O que observar no output
Cada linha deve somar 100% (1.0) e representar “de onde saiu” para “onde foi”.

In [ ]:
trans_counts = pd.crosstab(  # conta transições entre bucket atual e próximo bucket
    df_trans["bucket"],
    df_trans["bucket_next"]
).reindex(index=states, columns=states, fill_value=0)

trans_counts  # exibe matriz de contagens

### 4.1) Normalizar contagens para obter probabilidades

### O que vamos fazer
Dividir cada linha da matriz de contagens pelo total da linha.

### Por que isso importa em FP&A
Transforma contagem em regra de migração mensal (matriz P).

### O que observar no output
Linhas devem somar 1.0.

In [ ]:
p_base = trans_counts.div(trans_counts.sum(axis=1), axis=0)  # converte contagens em proporções por linha
p_base = p_base.fillna(0.0)  # trata eventuais linhas sem massa

p_base.loc["WriteOff", :] = 0.0  # zera linha de WriteOff para forçar estado absorvente
p_base.loc["WriteOff", "WriteOff"] = 1.0  # define WriteOff -> WriteOff = 100%

print("Soma das linhas da matriz P:")  # título para validar normalização
print(p_base.sum(axis=1))  # imprime soma de cada linha

### 4.2) Tabela formatada da matriz P

### O que vamos fazer
Mostrar P com percentuais para leitura executiva.

### Por que isso importa em FP&A
Facilita levar a matriz para slide e discussão de comitê.

### O que observar no output
Migrações críticas como `OK→Aten` e `Mau→WriteOff`.

In [ ]:
p_display = (p_base * 100).round(1).astype(str) + "%"  # formata probabilidades em percentual com 1 casa
p_display  # exibe tabela formatada da matriz P

## 5) Heatmap executivo da matriz P (Plotly)

### O que vamos fazer
Plotar a matriz de transição como heatmap.

### Por que isso importa em FP&A
Visual rápido de permanência e deterioração.

### O que observar no output
Células mais quentes indicam migrações mais prováveis.

In [ ]:
fig_heat = go.Figure(  # cria figura do heatmap da matriz de transição
    data=go.Heatmap(
        z=(p_base.values * 100),
        x=states,
        y=states,
        colorscale="YlOrRd",
        text=(p_base.values * 100).round(1),
        texttemplate="%{text:.1f}%"
    )
)
fig_heat.update_layout(title="PARTE B — Heatmap da matriz de transição P", xaxis_title="Próximo estado", yaxis_title="Estado atual")  # ajusta layout
fig_heat.show()  # exibe heatmap

### Como ler o heatmap
- Diagonal = permanência no mesmo bucket.
- `OK→Aten` e `Mau→WriteOff` são migrações críticas para risco.
- Célula mais “quente” = maior probabilidade de migração.

### KPIs de migração que FP&A monitora

### O que vamos fazer
Extrair 3 transições críticas diretamente da matriz P base.

### Por que isso importa em FP&A
- São os “drivers” de deterioração da carteira.
- Mudança nesses pontos impacta PDD e metas.

### O que observar no output
Probabilidades de `OK→Aten`, `Aten→Mau` e `Mau→WriteOff` em formato executivo.

In [ ]:
p_ok_aten = float(p_base.loc["OK", "Aten"])  # captura a probabilidade de migração de OK para Aten
p_aten_mau = float(p_base.loc["Aten", "Mau"])  # captura a probabilidade de migração de Aten para Mau
p_mau_wo = float(p_base.loc["Mau", "WriteOff"])  # captura a probabilidade de migração de Mau para WriteOff

kpi_trans = pd.DataFrame({  # monta tabela com as três transições críticas
    "transicao": ["OK→Aten", "Aten→Mau", "Mau→WriteOff"],
    "prob": [p_ok_aten, p_aten_mau, p_mau_wo]
})

kpi_trans["prob_pct"] = (kpi_trans["prob"] * 100).round(1)  # converte probabilidades para percentual
kpi_trans["prob_fmt"] = kpi_trans["prob_pct"].map(lambda x: f"{x:.1f}%")  # formata percentuais para leitura executiva

kpi_trans[["transicao", "prob_fmt"]]  # exibe tabela curta de KPIs críticos

**Leitura FP&A:** se `OK→Aten` sobe, é sinal macro piorando; se `Mau→WriteOff` sobe, a perda acelera.

### Gráfico executivo dos KPIs críticos

### O que vamos fazer
Plotar as 3 transições críticas em barras para comparação direta.

### Por que isso importa em FP&A
Esse painel vira artefato mensal de governança para comitê.

### O que observar no output
Altura das barras e ranking das migrações críticas.

In [ ]:
fig_kpi = px.bar(  # cria gráfico de barras para as transições críticas
    kpi_trans,
    x="transicao",
    y="prob",
    text="prob_fmt",
    title="PARTE B — Transições críticas (Matriz P base)"
)
fig_kpi.update_layout(xaxis_title="Transição", yaxis_title="Probabilidade")  # configura títulos dos eixos
fig_kpi.show()  # renderiza painel de KPIs de migração

## 6) Projeção 6 meses da distribuição por bucket

### O que vamos fazer
Montar `π0` no último mês e aplicar a matriz P mês a mês.

### Por que isso importa em FP&A
É a forma prática de projetar deterioração sem matemática pesada.

### O que observar no output
Distribuição projetada de 0 a 6 meses para cada bucket.

In [ ]:
pi0_ead = df_last.groupby("bucket")["ead"].sum().reindex(states, fill_value=0.0)  # soma EAD por bucket no último mês
pi0_ead = pi0_ead / pi0_ead.sum()  # converte para participação de EAD por bucket

pi0_contract = df_last["bucket"].value_counts(normalize=True).reindex(states, fill_value=0.0)  # participação por quantidade de contratos

pi0_table = pd.DataFrame({  # monta tabela comparativa de ponto inicial
    "bucket": states,
    "% contratos": pi0_contract.values,
    "% EAD": pi0_ead.values
})

pi0_table  # exibe composição inicial

### 6.1) Tabela formatada de π0 (contratos vs EAD)

### O que vamos fazer
Formatar percentuais para leitura rápida.

### Por que isso importa em FP&A
A projeção principal usará `% EAD`, mas é útil comparar com `% contratos`.

### O que observar no output
Diferenças entre visão de volume e visão financeira.

In [ ]:
pi0_display = pi0_table.copy()  # cria cópia para formatação visual
pi0_display["% contratos"] = (pi0_display["% contratos"] * 100).round(1).astype(str) + "%"  # formata percentual de contratos
pi0_display["% EAD"] = (pi0_display["% EAD"] * 100).round(1).astype(str) + "%"  # formata percentual de EAD

pi0_display  # mostra tabela formatada

### 6.2) Aplicar P por 6 passos

### O que vamos fazer
Começar em `π0` e aplicar `π_{t+1} = π_t dot P` por 6 meses.

### Por que isso importa em FP&A
Esse processo gera a trilha projetada da carteira por bucket.

### O que observar no output
Tabela com meses 0..6 e participações por bucket.

In [ ]:
pi_current = pi0_ead.values.copy()  # inicia vetor com distribuição de EAD no mês 0
proj_rows = [{"mes": 0, "OK": pi_current[0], "Aten": pi_current[1], "Mau": pi_current[2], "WriteOff": pi_current[3]}]  # guarda ponto inicial

for step in range(1, 7):  # aplica matriz P seis vezes (meses 1 a 6)
    pi_current = pi_current.dot(p_base.values)  # atualiza distribuição para o próximo mês
    proj_rows.append({"mes": step, "OK": pi_current[0], "Aten": pi_current[1], "Mau": pi_current[2], "WriteOff": pi_current[3]})  # guarda resultado do mês

proj_base = pd.DataFrame(proj_rows)  # converte resultados em DataFrame
proj_base  # exibe tabela numérica da projeção

### 6.3) Tabela formatada da projeção por bucket

### O que vamos fazer
Formatar projeção com percentuais para apresentação.

### Por que isso importa em FP&A
Tabela pronta para slide/comitê sem poluição visual.

### O que observar no output
Evolução de `OK` vs buckets piores ao longo dos meses.

In [ ]:
proj_display = proj_base.copy()  # cria cópia para formatação
for col in states:  # percorre buckets para formatar percentual
    proj_display[col] = (proj_display[col] * 100).round(1).astype(str) + "%"

proj_display  # exibe projeção formatada

### 6.4) Área empilhada da distribuição projetada (Plotly)

### O que vamos fazer
Plotar composição projetada dos buckets de 0 a 6 meses.

### Por que isso importa em FP&A
Mostra rapidamente tendência de deterioração ou estabilização.

### O que observar no output
Participação de buckets críticos ao longo do horizonte.

In [ ]:
proj_melt = proj_base.melt(id_vars="mes", value_vars=states, var_name="bucket", value_name="pct")  # transforma tabela para formato longo

fig_area = px.area(  # cria gráfico de área empilhada da projeção
    proj_melt,
    x="mes",
    y="pct",
    color="bucket",
    title="PARTE C — Distribuição projetada por bucket (0–6 meses)"
)
fig_area.update_layout(xaxis_title="Mês projetado", yaxis_title="Participação de EAD")  # ajusta eixos
fig_area.show()  # exibe gráfico

## 7) Converter projeção em PDD projetada (R$)

### O que vamos fazer
Aplicar LGD por bucket sobre saldos projetados para obter PDD em cada mês.

### Por que isso importa em FP&A
Transforma distribuição de risco em impacto financeiro direto no DRE.

### O que observar no output
PDD tende a crescer se aumentar participação de buckets piores.

In [ ]:
lgd_map = {"OK": 0.00, "Aten": 0.05, "Mau": 0.25, "WriteOff": 0.80}  # define LGD simplificada por bucket
ead_total = df_last["ead"].sum()  # calcula EAD total no último mês observado

print(f"EAD total (mês base): R$ {ead_total:,.0f}")  # imprime base financeira para projeção
print("LGD por bucket:", lgd_map)  # imprime dicionário de LGD usado no cálculo

### 7.1) Calcular PDD para meses 0..6

### O que vamos fazer
Converter participação por bucket em saldo (R$) e depois em PDD (R$).

### Por que isso importa em FP&A
É o indicador final para provisão e planejamento financeiro.

### O que observar no output
Tabela com PDD mensal e saldos por bucket.

In [ ]:
pdd_rows = []  # inicia lista para resultados de PDD

for _, row in proj_base.iterrows():  # percorre cada mês projetado
    saldo_ok = ead_total * row["OK"]  # calcula saldo projetado em OK
    saldo_aten = ead_total * row["Aten"]  # calcula saldo projetado em Aten
    saldo_mau = ead_total * row["Mau"]  # calcula saldo projetado em Mau
    saldo_wo = ead_total * row["WriteOff"]  # calcula saldo projetado em WriteOff

    pdd_t = (saldo_ok * lgd_map["OK"] + saldo_aten * lgd_map["Aten"] + saldo_mau * lgd_map["Mau"] + saldo_wo * lgd_map["WriteOff"])  # soma PDD do mês
    pdd_rows.append({"mes": int(row["mes"]), "saldo_OK": saldo_ok, "saldo_Aten": saldo_aten, "saldo_Mau": saldo_mau, "saldo_WriteOff": saldo_wo, "PDD_R$": pdd_t})  # guarda linha do mês

pdd_base = pd.DataFrame(pdd_rows)  # transforma lista em tabela
pdd_base  # mostra tabela numérica base

### 7.2) Tabela formatada de PDD

### O que vamos fazer
Formatar valores monetários para leitura executiva.

### Por que isso importa em FP&A
PDD em R$ precisa estar legível para decisão rápida.

### O que observar no output
Evolução do valor projetado de provisão.

In [ ]:
pdd_display = pdd_base[["mes", "PDD_R$"]].copy()  # seleciona colunas essenciais para apresentação
pdd_display["PDD_R$"] = pdd_display["PDD_R$"].round(0).map(lambda x: f"R$ {x:,.0f}")  # formata PDD em moeda

pdd_display  # exibe tabela formatada de PDD

### 7.3) Linha da PDD projetada (Plotly)

### O que vamos fazer
Plotar evolução de PDD em R$ de 0 a 6 meses.

### Por que isso importa em FP&A
Gráfico facilita discutir tendência de provisão com gestão e comitê.

### O que observar no output
Inclinação da linha indica aceleração ou estabilização da PDD.

In [ ]:
fig_pdd = px.line(  # cria gráfico de linha da PDD projetada
    pdd_base,
    x="mes",
    y="PDD_R$",
    markers=True,
    title="PARTE D — PDD projetada em R$ (0–6 meses)"
)
fig_pdd.update_layout(xaxis_title="Mês projetado", yaxis_title="PDD (R$)")  # ajusta títulos dos eixos
fig_pdd.show()  # exibe gráfico

**Leitura FP&A:** a PDD cresce quando aumenta participação dos buckets com LGD maior, principalmente `Mau` e `WriteOff`.

## 8) Stress test manual (sem usar scenario)

### O que vamos fazer
Ajustar duas transições críticas na matriz P para cenário mais estressado.

### Por que isso importa em FP&A
Pequenas mudanças de migração podem gerar grande variação na provisão.

### O que observar no output
Comparação direta de PDD base vs PDD stress.

In [ ]:
p_stress = p_base.copy()  # copia matriz base para aplicar stress

p_stress.loc["OK", "OK"] = p_stress.loc["OK", "OK"] - 0.02  # reduz permanência em OK em 2pp
p_stress.loc["OK", "Aten"] = p_stress.loc["OK", "Aten"] + 0.02  # aumenta OK->Aten em 2pp

p_stress.loc["Mau", "Mau"] = p_stress.loc["Mau", "Mau"] - 0.03  # reduz permanência em Mau em 3pp
p_stress.loc["Mau", "WriteOff"] = p_stress.loc["Mau", "WriteOff"] + 0.03  # aumenta Mau->WriteOff em 3pp

p_stress.loc["OK", :] = p_stress.loc["OK", :] / p_stress.loc["OK", :].sum()  # re-normaliza linha OK para somar 100%
p_stress.loc["Mau", :] = p_stress.loc["Mau", :] / p_stress.loc["Mau", :].sum()  # re-normaliza linha Mau para somar 100%

### Heatmap de ΔP (stress − base)

### O que vamos fazer
Calcular `ΔP = P_stress − P_base` para mostrar exatamente onde mexemos no stress.

### Por que isso importa em FP&A
Esse gráfico reforça governança: o comitê enxerga quais transições foram alteradas.

### O que observar no output
Valores positivos indicam aumento de migração no cenário stress.

In [ ]:
p_delta = p_stress - p_base  # calcula diferença entre matriz stress e matriz base

fig_delta = go.Figure(  # cria heatmap de diferença com escala divergente
    data=go.Heatmap(
        z=p_delta.values,
        x=states,
        y=states,
        colorscale="RdBu",
        zmid=0,
        text=p_delta.values,
        texttemplate="%{text:+.2f}"
    )
)
fig_delta.update_layout(title="PARTE E — Heatmap de ΔP (stress - base)", xaxis_title="Próximo estado", yaxis_title="Estado atual")  # configura layout
fig_delta.show()  # exibe heatmap com alterações aplicadas no stress

### 8.1) Projetar 6 meses com matriz stress

### O que vamos fazer
Repetir a projeção com `P_stress`.

### Por que isso importa em FP&A
Permite medir sensibilidade da carteira a deteriorações específicas.

### O que observar no output
Trajetória de buckets mais pessimista que o cenário base.

In [ ]:
pi_current = pi0_ead.values.copy()  # reinicia vetor no mesmo ponto de partida
proj_rows_stress = [{"mes": 0, "OK": pi_current[0], "Aten": pi_current[1], "Mau": pi_current[2], "WriteOff": pi_current[3]}]  # guarda mês zero

for step in range(1, 7):  # aplica matriz stress por seis meses
    pi_current = pi_current.dot(p_stress.values)  # atualiza distribuição usando P_stress
    proj_rows_stress.append({"mes": step, "OK": pi_current[0], "Aten": pi_current[1], "Mau": pi_current[2], "WriteOff": pi_current[3]})  # salva resultado do mês

proj_stress = pd.DataFrame(proj_rows_stress)  # converte projeção stress em DataFrame
proj_stress  # exibe projeção numérica stress

### 8.2) Calcular PDD no cenário stress

### O que vamos fazer
Aplicar o mesmo LGD e EAD total sobre a projeção stress.

### Por que isso importa em FP&A
Mantemos comparabilidade justa com o cenário base.

### O que observar no output
Valor de PDD stress por mês.

In [ ]:
pdd_rows_stress = []  # inicia lista de resultados stress

for _, row in proj_stress.iterrows():  # percorre meses da projeção stress
    saldo_ok = ead_total * row["OK"]  # saldo stress em OK
    saldo_aten = ead_total * row["Aten"]  # saldo stress em Aten
    saldo_mau = ead_total * row["Mau"]  # saldo stress em Mau
    saldo_wo = ead_total * row["WriteOff"]  # saldo stress em WriteOff

    pdd_t = (saldo_ok * lgd_map["OK"] + saldo_aten * lgd_map["Aten"] + saldo_mau * lgd_map["Mau"] + saldo_wo * lgd_map["WriteOff"])  # calcula PDD stress do mês
    pdd_rows_stress.append({"mes": int(row["mes"]), "PDD_R$_stress": pdd_t})  # salva valor do mês

pdd_stress = pd.DataFrame(pdd_rows_stress)  # monta tabela final do stress
pdd_stress  # exibe PDD stress

### 8.3) Comparar PDD base vs stress

### O que vamos fazer
Consolidar as duas trilhas de PDD e calcular diferença em R$.

### Por que isso importa em FP&A
Mostra impacto financeiro direto de mudanças em transições críticas.

### O que observar no output
ΔPDD no mês 6 para discussão de sensibilidade.

In [ ]:
pdd_compare = pdd_base[["mes", "PDD_R$"]].merge(pdd_stress, on="mes", how="left")  # junta cenário base e stress
pdd_compare["Delta_R$"] = pdd_compare["PDD_R$_stress"] - pdd_compare["PDD_R$"]  # calcula variação absoluta

pdd_compare_display = pdd_compare.copy()  # cria cópia para visualização
pdd_compare_display["PDD_R$"] = pdd_compare_display["PDD_R$"].round(0).map(lambda x: f"R$ {x:,.0f}")  # formata base
pdd_compare_display["PDD_R$_stress"] = pdd_compare_display["PDD_R$_stress"].round(0).map(lambda x: f"R$ {x:,.0f}")  # formata stress
pdd_compare_display["Delta_R$"] = pdd_compare_display["Delta_R$"].round(0).map(lambda x: f"R$ {x:,.0f}")  # formata delta

pdd_compare_display  # exibe comparação formatada

### 8.4) Plot base vs stress (Plotly)

### O que vamos fazer
Desenhar duas linhas e destacar o ΔPDD no mês 6.

### Por que isso importa em FP&A
Visualiza rapidamente o custo da deterioração de migração.

### O que observar no output
Distância entre linhas e anotação de delta no horizonte final.

In [ ]:
delta_m6 = pdd_compare.loc[pdd_compare["mes"] == 6, "Delta_R$"].iloc[0]  # captura delta no mês 6 para anotação

fig_stress = go.Figure()  # inicia figura comparativa
fig_stress.add_trace(go.Scatter(x=pdd_compare["mes"], y=pdd_compare["PDD_R$"], mode="lines+markers", name="PDD Base"))  # adiciona linha base
fig_stress.add_trace(go.Scatter(x=pdd_compare["mes"], y=pdd_compare["PDD_R$_stress"], mode="lines+markers", name="PDD Stress"))  # adiciona linha stress

fig_stress.add_annotation(x=6, y=pdd_compare.loc[pdd_compare["mes"] == 6, "PDD_R$_stress"].iloc[0], text=f"Δ mês 6: R$ {delta_m6:,.0f}", showarrow=True)  # anota delta final
fig_stress.update_layout(title="PARTE E — PDD Base vs Stress (0–6 meses)", xaxis_title="Mês projetado", yaxis_title="PDD (R$)")  # ajusta layout
fig_stress.show()  # exibe gráfico

### Destaque do impacto financeiro no horizonte (M+6)

### O que vamos fazer
Agora vamos destacar no gráfico o impacto final: `ΔPDD = PDD_stress - PDD_base` no mês 6.

### Por que isso importa em FP&A
O valor de M+6 é um resumo executivo direto para decisão em comitê.

### O que observar no output
Veja a anotação no ponto final da curva stress com o valor em R$.

In [ ]:
pdd_base_m6 = float(pdd_compare.loc[pdd_compare["mes"] == 6, "PDD_R$"].iloc[0])  # captura PDD base no mês 6
pdd_stress_m6 = float(pdd_compare.loc[pdd_compare["mes"] == 6, "PDD_R$_stress"].iloc[0])  # captura PDD stress no mês 6
delta_pdd_m6 = pdd_stress_m6 - pdd_base_m6  # calcula delta financeiro no horizonte

delta_pdd_m6_fmt = f"R$ {delta_pdd_m6:,.0f}"  # formata delta com separador de milhar para leitura executiva

fig_stress.add_annotation(  # adiciona anotação explícita do delta no mês 6
    x=6,
    y=pdd_stress_m6,
    text=f"ΔPDD (M+6) = {delta_pdd_m6_fmt}",
    showarrow=True,
    arrowhead=2,
    ax=50,
    ay=-35
)
fig_stress.show()  # re-renderiza gráfico com anotação explícita de M+6

### Insight FP&A
Pequena mudança em migração crítica (`OK→Aten` e `Mau→WriteOff`) pode gerar impacto relevante na provisão em poucos meses.

## 9) Backtest simples (opcional)

### O que vamos fazer
Treinar P nos 12 primeiros meses e comparar previsão de 3 meses com o real.

### Por que isso importa em FP&A
É um termômetro rápido para saber se a matriz está razoável fora da amostra de treino.

### O que observar no output
Erro absoluto por bucket entre distribuição prevista e real.

In [ ]:
months = sorted(df["month"].unique())  # ordena meses disponíveis
m12 = months[11]  # define mês de corte para treino (12º mês)
m15 = months[14]  # define horizonte de comparação (mês 15)

df_train = df[df["month"] <= m12].copy()  # usa primeiros 12 meses para treino da matriz
start_df = df[df["month"] == m12].copy()  # define distribuição inicial para projeção de 3 meses
real_m15 = df[df["month"] == m15].copy()  # define distribuição real no horizonte de comparação

print(f"Treino até: {m12.date()} | Comparação real em: {m15.date()}")  # imprime marcos temporais do backtest

### 9.1) Projeção de 3 meses usando matriz de treino

### O que vamos fazer
Montar `P_train`, projetar 3 passos e comparar com distribuição real do mês 15.

### Por que isso importa em FP&A
Mostra rapidamente a aderência do modelo de migração.

### O que observar no output
Tabela de erro absoluto por bucket.

In [ ]:
train_trans = df_train[["contract_id", "month", "bucket"]].copy()  # recorta dados de treino para transições
train_trans["bucket_next"] = train_trans.groupby("contract_id")["bucket"].shift(-1)  # cria próximo bucket no treino
train_trans = train_trans.dropna(subset=["bucket_next"]).copy()  # remove linhas sem próximo bucket

counts_train = pd.crosstab(train_trans["bucket"], train_trans["bucket_next"]).reindex(index=states, columns=states, fill_value=0)  # conta transições no treino
p_train = counts_train.div(counts_train.sum(axis=1), axis=0).fillna(0.0)  # normaliza para matriz de probabilidades
p_train.loc["WriteOff", :] = 0.0  # força linha WriteOff para estado absorvente
p_train.loc["WriteOff", "WriteOff"] = 1.0  # define transição absorvente em WriteOff

pi_start = start_df.groupby("bucket")["ead"].sum().reindex(states, fill_value=0.0)  # captura distribuição de EAD no mês de partida
pi_start = (pi_start / pi_start.sum()).values  # converte para vetor de participação

pi_pred_3m = pi_start.dot(np.linalg.matrix_power(p_train.values, 3))  # projeta três passos à frente
pi_real_3m = real_m15.groupby("bucket")["ead"].sum().reindex(states, fill_value=0.0)  # captura distribuição real de EAD no mês alvo
pi_real_3m = (pi_real_3m / pi_real_3m.sum()).values  # converte para vetor de participação real

### 9.2) Tabela de erro absoluto por bucket

### O que vamos fazer
Comparar previsto vs real no horizonte de 3 meses.

### Por que isso importa em FP&A
Erro por bucket ajuda a calibrar janela de atualização da matriz P.

### O que observar no output
Buckets com erro absoluto maior merecem monitoramento.

In [ ]:
bt_table = pd.DataFrame({  # monta tabela de comparação do backtest
    "bucket": states,
    "previsto_3m": pi_pred_3m,
    "real_3m": pi_real_3m
})
bt_table["erro_abs"] = (bt_table["previsto_3m"] - bt_table["real_3m"]).abs()  # calcula erro absoluto por bucket

bt_display = bt_table.copy()  # cria cópia para formatação
for col in ["previsto_3m", "real_3m", "erro_abs"]:  # formata colunas percentuais
    bt_display[col] = (bt_display[col] * 100).round(1).astype(str) + "%"

bt_display  # exibe tabela de erro formatada

### 9.3) Barras real vs previsto (3 meses)

### O que vamos fazer
Mostrar lado a lado a distribuição prevista e real por bucket.

### Por que isso importa em FP&A
Facilita leitura visual da aderência do backtest.

### O que observar no output
Diferenças de altura entre barras indicam buckets com maior desvio.

In [ ]:
bt_plot = bt_table.melt(id_vars="bucket", value_vars=["previsto_3m", "real_3m"], var_name="serie", value_name="pct")  # transforma dados para barras agrupadas

fig_bt = px.bar(  # cria gráfico real vs previsto por bucket
    bt_plot,
    x="bucket",
    y="pct",
    color="serie",
    barmode="group",
    title="PARTE F — Backtest (horizonte 3 meses): real vs previsto"
)
fig_bt.update_layout(xaxis_title="Bucket", yaxis_title="Participação")  # ajusta eixos
fig_bt.show()  # exibe gráfico

## 10) Conclusões executivas + Checklist FP&A

### Conclusões executivas
1. Markov resume migração por buckets em uma matriz P objetiva.
2. O heatmap mostra rapidamente onde a carteira está deteriorando.
3. A projeção 6m entrega saldo por bucket mês a mês.
4. LGD converte a projeção para PDD em R$.
5. Stress em transições críticas mostra sensibilidade do DRE.
6. A matriz P deve ser monitorada e atualizada com mudança de regime.
7. Painel de transições críticas + heatmap de ΔP reforçam governança para comitê.

### Checklist FP&A
- Estados (buckets) definidos e estáveis.
- Matriz P atualizada com janela recente.
- Heatmap mensal para migrações críticas.
- Projeção curta (3–6 meses) para planejamento.
- Stress test padronizado para comitê.
- Monitoramento de drift (`OK→Aten`, `Mau→WriteOff`).